In [1]:
!pip install duckdb --upgrade

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 137.4 MB/s eta 0:00:0000:0100:01


Register an App if running outside Fabric to get a token
https://fivetran.com/docs/destinations/onelake/setup-guide

In [2]:
import duckdb

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 4, Finished, Available, Finished)

In [3]:
duckdb.sql(" install azure ;force install delta from core_nightly ; ")

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 5, Finished, Available, Finished)

In [4]:
try:
    token = notebookutils.credentials.getToken('storage')
except:
    !pip install -q azure-identity
    from   azure.identity   import ClientSecretCredential
    import configparser
    config = configparser.ConfigParser()
    config.read("C:/KV/variable.ini")
    credential = ClientSecretCredential(
                    client_id     =config.get("myvars", "appId"),
                    client_secret =config.get("myvars", "secret") , 
                    tenant_id     =config.get("myvars", "tenantId")
                    )
    token =       credential.get_token("https://storage.azure.com/.default").token

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 6, Finished, Available, Finished)

In [5]:
def attach_lakehouse(workspace,lakehouse,token):
  duckdb.sql(f"""          
    load azure ;
    CREATE or replace SECRET secret4 ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}');
    set enable_object_cache = true 
     """)
  tables = duckdb.sql(f""" SELECT distinct(split_part(file, '/', 7)) as tables FROM glob
                      ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/*/_delta_log/*.json")
                    """).df()['tables'].tolist()
  for tbl in tables:
    print(tbl)
    duckdb.sql(f""" FROM delta_scan("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{tbl}")  """).to_view(tbl)
 

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 7, Finished, Available, Finished)

In [6]:
# use Workspace and lakehouse name, token either using notebookutils inside Fabric
#  or register an App for access outside Fabric
attach_lakehouse("aemodev","storage",token)

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 8, Finished, Available, Finished)

states
duid
mstdatetime
price
scada
calendar


In [7]:
duckdb.sql(""" 
           select Region, sum(INITIALMW)/12 as MWH from scada s
            join duid d
            on s.DUID = d.DUID
            where settlementdate > '2024-09-01'
            GROUP BY ALL
            order by MWH desc
           """)

StatementMeta(, 756b45bf-d9b7-40a5-9c9b-87b032441aca, 9, Finished, Available, Finished)

┌─────────┬────────────────────┐
│ Region  │        MWH         │
│ varchar │       double       │
├─────────┼────────────────────┤
│ NSW1    │ 1709142.3626528624 │
│ QLD1    │ 1625296.9488299442 │
│ VIC1    │ 1451881.2432481016 │
│ SA1     │  330328.0311883463 │
│ TAS1    │  293893.6644291807 │
│ WA1     │  41197.83914295663 │
└─────────┴────────────────────┘